# Concept Notebook
Start with a blank default application and programmatically add views and data to those views

### Use Case
Progammatically load a single FITS file, e.g. for 2d spectroscopic data, into an "image" viewer.  Or a 1d spectrum into a spectrum viewer.  For cases that do not fit into a given pre-made configuration. 

### MAST Use Case
MAST auto-generates notebooks that when users run, needs to download the data, create the relevant jdaviz application / viewers, and load the data by default. One click run gets the user back to where they were on the web.

In [10]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [11]:
from jdaviz.app import Application

In [12]:
# some 2d spectroscopic data
data = '/Users/bcherinka/Work/jwst/test/jw00626-o030_s00000_nirspec_f170lp-g235m_s2d.fits'

### Option 1 - Create a default viewer with the UI and programmatically add data
In this option, we create jdaviz with a default configuration, and load an "Image 2D" viewer via the user interface, (viewer creator plugin).  We then attempt to add data to the viewer programmatically.  

In [13]:
app = Application('default')
app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

I know I first need to load the data

In [14]:
app.load_data(data)

# running this twice loads the same data with the same labels.  Downstream analysis/plugin calls raise errors on duplicate and 
# ambiguous data labels.  Running this twice on the same data should either not re-load it or load it with a different unique
# data label

/Users/bcherinka/anaconda3/envs/havokve/lib/python3.8/site-packages/glue/core/data_factories/fits.py:152: UserWarning: Dropping column 'ASDF_METADATA' since it is not 1-dimensional
  warnings.warn("Dropping column '{0}' since it is not 1-dimensional".format(column_name))


In [15]:
# what label did my data get?  dig around in docs to understand data_collection.  How do I define custom data labels?
app.data_collection

# want - option to easily label or relabel incoming data.  Either as input in app.load_data or a new function, e.g. app.relabel_data()
# app.add_data requires data to be in a special format

DataCollection (5 data sets)
	  0: jw00626-o030_s00000_nirspec_f170lp-g235m_s2d[SCI]
	  1: jw00626-o030_s00000_nirspec_f170lp-g235m_s2d[WHT]
	  2: jw00626-o030_s00000_nirspec_f170lp-g235m_s2d[CON]
	  3: jw00626-o030_s00000_nirspec_f170lp-g235m_s2d[HDRTAB]
	  4: jw00626-o030_s00000_nirspec_f170lp-g235m_s2d[ASDF]

In [16]:
# get a label
label = app.data_collection[0].label
label

'jw00626-o030_s00000_nirspec_f170lp-g235m_s2d[SCI]'

I want to load the data into the viewer.  I don't know the name of the viewer I created.  How do I get a list of current viewer references?

In [17]:
# naively try to get "image-viewer". ; throws error
app.get_viewer('image-viewer')

# also try 
#app.get_viewer('g-image-viewer')
#app.get_viewer('flux-viewer')

# want - to return a list of all viewer reference names for currently loaded viewers.  Pick from the list and add data.
# app.list_viewers()

TypeError: 'NoneType' object is not subscriptable

In [18]:
# let's try to add data anyways - throws same error
app.add_data_to_viewer('image-viewer', data, ext='SCI')

TypeError: 'NoneType' object is not subscriptable

How can I access and inspect the available viewers?  

In [19]:
# look in the viewer registry
from jdaviz.core.registries import tool_registry, viewer_registry
viewer_registry

In [20]:
# I can find all viewers in the registry. But these dict keys are not reference names.  
viewer_registry.members

{'cubeviz-image-viewer': {'label': 'Image 2D (CubeViz)',
  'cls': jdaviz.configs.cubeviz.plugins.viewers.CubeVizImageView},
 'g-profile-viewer': {'label': 'Profile 1D',
  'cls': glue_jupyter.bqplot.profile.viewer.BqplotProfileView},
 'g-image-viewer': {'label': 'Image 2D',
  'cls': glue_jupyter.bqplot.image.viewer.BqplotImageView},
 'g-table-viewer': {'label': 'Table',
  'cls': glue_jupyter.table.viewer.TableViewer},
 'specviz-profile-viewer': {'label': 'Profile 1D (Specviz)',
  'cls': jdaviz.configs.specviz.plugins.viewers.SpecvizProfileView},
 'mosviz-profile-viewer': {'label': 'Profile 1D (MOSViz)',
  'cls': jdaviz.configs.mosviz.plugins.viewers.MOSVizProfileView},
 'mosviz-image-viewer': {'label': 'Image 2D (MOSViz)',
  'cls': jdaviz.configs.mosviz.plugins.viewers.MOSVizImageView},
 'mosviz-profile-2d-viewer': {'label': 'Spectrum 2D (MOSViz)',
  'cls': jdaviz.configs.mosviz.plugins.viewers.MOSVizProfile2DView},
 'mosviz-table-viewer': {'label': 'Table (MOSViz)',
  'cls': jdaviz.con

In [21]:
# oh I see my "Image 2D" == "g-image-viewer".  That must be the reference name.  Let me try that, to get the viewer.
app.get_viewer('g-image-viewer')

TypeError: 'NoneType' object is not subscriptable

Now I'm lost and don't know how to look up my viewer.  I can dig around in hidden attributes.

In [22]:
# look up the viewer store.  I see my viewer with a viewer id. 
app._viewer_store

{'6501402f-bb03-4fd5-b8da-1b4241daf66b': <glue_jupyter.bqplot.image.viewer.BqplotImageView at 0x7ffa6a9fd400>}

In [23]:
# examine the application stack of items
app.state.stack_items

<CallbackList with 1 elements>

In [24]:
# loop over all the items in the application stack and print the data
for i in app.state.stack_items:
    print('item', i.keys())
    for viewer in i.get('viewers'):
        print('viewer', viewer.keys())
        for kk, vv in viewer.items():
            print(kk,vv)

item dict_keys(['id', 'container', 'children', 'viewers'])
viewer dict_keys(['id', 'name', 'widget', 'tools', 'layer_options', 'viewer_options', 'selected_data_items', 'collapse', 'reference', 'tab'])
id 6501402f-bb03-4fd5-b8da-1b4241daf66b
name Unnamed Viewer
widget IPY_MODEL_2000302c0e344576b421f3f18cf8ba6e
tools IPY_MODEL_f4c490d6cc2e4b31893e6c2b4c4f5062
layer_options IPY_MODEL_75dae667d8454464a286b39ff444a678
viewer_options IPY_MODEL_4772c7d7dddd428a9533f6877ac18183
selected_data_items <CallbackList with 0 elements>
collapse True
reference None
tab 0


In [43]:
# now I have the viewer, I think.  The repr here should be improved to more obvious.
viewer

<CallbackDict with 10 elements>

In [25]:
# get the viewer by id
id = viewer['id']

In [26]:
# try to add data by id.  Still same error as above.
app.add_data_to_viewer(id, label)

TypeError: 'NoneType' object is not subscriptable

In [27]:
# I notice in the stack that no reference name is indicated.  Let's add one.
view = app._viewer_item_by_id(id)
view['reference'] = 'image-viewer'
view

<CallbackDict with 10 elements>

In [28]:
# now I can add data to the viewer.  Success.
app.add_data_to_viewer('image-viewer', data, ext='SCI')

In [29]:
# can I access the data back out? 
app.get_data_from_viewer('image-viewer')

AttributeError: 'BqplotImageView' object has no attribute 'default_class'

### Option 2 - Create the viewer programmatically and also load some data
In this option, we create a blank jdaviz application with default configuration, and attempt to create a "Image 2D" viewer programmatically, and add data to it. 

In [30]:
app = Application()
app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

I don't know what viewers are available to add.  How can I find this out?

In [31]:
# how do I know what viewers are available to add?  Use the viewer registry I dug around in the code to find.
viewer_registry.members

# want - option to list/dict the available options for creating different viewers; returns a reference name, label and class.
# i.e. expose the registry members in a user-friendly format
# app.list_viewer_types()

{'cubeviz-image-viewer': {'label': 'Image 2D (CubeViz)',
  'cls': jdaviz.configs.cubeviz.plugins.viewers.CubeVizImageView},
 'g-profile-viewer': {'label': 'Profile 1D',
  'cls': glue_jupyter.bqplot.profile.viewer.BqplotProfileView},
 'g-image-viewer': {'label': 'Image 2D',
  'cls': glue_jupyter.bqplot.image.viewer.BqplotImageView},
 'g-table-viewer': {'label': 'Table',
  'cls': glue_jupyter.table.viewer.TableViewer},
 'specviz-profile-viewer': {'label': 'Profile 1D (Specviz)',
  'cls': jdaviz.configs.specviz.plugins.viewers.SpecvizProfileView},
 'mosviz-profile-viewer': {'label': 'Profile 1D (MOSViz)',
  'cls': jdaviz.configs.mosviz.plugins.viewers.MOSVizProfileView},
 'mosviz-image-viewer': {'label': 'Image 2D (MOSViz)',
  'cls': jdaviz.configs.mosviz.plugins.viewers.MOSVizImageView},
 'mosviz-profile-2d-viewer': {'label': 'Spectrum 2D (MOSViz)',
  'cls': jdaviz.configs.mosviz.plugins.viewers.MOSVizProfile2DView},
 'mosviz-table-viewer': {'label': 'Table (MOSViz)',
  'cls': jdaviz.con

ok, so I know I want an Image 2d viewer, but don't know how to, or cannot, create one programmatically.

In [32]:
# want - option to create a viewer by reference name or label.  The input should check against the formatted list from the registry.
# app.create_viewer('image-viewer') or app.create_viewer('Image 2D')

# hack to create a new viewer programmatically using code from the Viewer Creator plugin
from jdaviz.core.events import NewViewerMessage, AddViewerMessage
viewer_cls = viewer_registry.members['g-image-viewer']['cls']
new_viewer_message = NewViewerMessage(viewer_cls, data=None, sender=app)
app.hub.broadcast(new_viewer_message)

In [33]:
# ok, I created a viewer.  Now have same problem as option 1
app.load_data(data)

/Users/bcherinka/anaconda3/envs/havokve/lib/python3.8/site-packages/glue/core/data_factories/fits.py:152: UserWarning: Dropping column 'ASDF_METADATA' since it is not 1-dimensional
  warnings.warn("Dropping column '{0}' since it is not 1-dimensional".format(column_name))


In [34]:
# look for the viewer in the store
app._viewer_store

{'c3906bad-0c1f-4e80-8be2-83984a01d3ed': <glue_jupyter.bqplot.image.viewer.BqplotImageView at 0x7ffa6ac45c10>}

In [35]:
# attempt to add the data with naive reference name
app.add_data_to_viewer('image-viewer', data, ext='SCI') 

TypeError: 'NoneType' object is not subscriptable

In [36]:
# grab the viewer information
id = next(iter(app._viewer_store))
view = app._viewer_item_by_id(id)
for k, v in view.items():
    print(k,v)

id c3906bad-0c1f-4e80-8be2-83984a01d3ed
name Unnamed Viewer
widget IPY_MODEL_288b6451c3784849a16055ef99d44e41
tools IPY_MODEL_b01dcb73142d4970a285fe0f48a3d39d
layer_options IPY_MODEL_a4ee8df437514ba784c3342839b0fa63
viewer_options IPY_MODEL_19604f2888394ef195438de35ea9134e
selected_data_items <CallbackList with 0 elements>
collapse True
reference None


In [37]:
# add valid reference name
view['reference'] = 'image-viewer'

In [38]:
# re-attempt to add the data.  success
app.add_data_to_viewer('image-viewer', data, ext='SCI') 

### Option 3 - Create custom Helper and define a custom configuration
This option attempts to create a custom helper that represents my 2d spectroscopic fits file, with a single "Image 2D viewer".  I define a new custom configuration with the viewer I need.

In [39]:
from jdaviz.core.helpers import ConfigHelper
from jdaviz.core.config import get_configuration

# get a default configuration
config = get_configuration('default')

# create a viewer area and viewer
config['viewer_area'] = [{'container': 'col', 'children': [{'container':'row', 'viewers':[]}]}]
viewers = {'viewers': [{'name': 'Image', 'plot': 'g-image-viewer', 'reference': 'image-viewer'}]}

# update the default config
config['viewer_area'][0]['children'][0].update(viewers)

config


# want - easier mechanism for adding viewers into a custom configuration.  Given a custom config, and a viewer 
# reference name or label, return a properly formatted new config
# add_viewer_to_config(config, 'image-viewer')  or add_viewer_to_config(config, ['image-viewer', 'spectrum-viewer'])

{'settings': {'visible': {'menu_bar': False,
   'toolbar': True,
   'tray': True,
   'tab_headers': True},
  'context': {'notebook': {'max_height': '600px'}}},
 'toolbar': ['g-data-tools', 'g-viewer-creator', 'g-subset-tools'],
 'tray': ['g-gaussian-smooth'],
 'viewer_area': [{'container': 'col',
   'children': [{'container': 'row',
     'viewers': [{'name': 'Image',
       'plot': 'g-image-viewer',
       'reference': 'image-viewer'}]}]}]}

In [40]:
# set my custom config object to the default configuration.  I could also just use Application(config) so 
# what does the ConfigHelper gain me, the user?  Would be nice to have default data loading. 
class ImViz(ConfigHelper):
    _default_configuration = config

im = ImViz()
im.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [41]:
im.load_data(data)

/Users/bcherinka/anaconda3/envs/havokve/lib/python3.8/site-packages/glue/core/data_factories/fits.py:152: UserWarning: Dropping column 'ASDF_METADATA' since it is not 1-dimensional
  warnings.warn("Dropping column '{0}' since it is not 1-dimensional".format(column_name))


In [42]:
# this works cleanly since I defined a proper reference viewer in my configuration
im.app.add_data_to_viewer('image-viewer', data, ext='SCI')

In [ ]:
# it would be nice if some convienence for creating data parses could live in the ConfigHelper or easily 
# decorate / attach to the new helper class